## Gait Video Study 
### From the camera calibration intrinsic and extrinsic matrix, computing the required scale subject wise to compute 3D from 2D coordinates

In [1]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\CameraCalibration\\'#1April\\mp4\\'

### Camera 1 - Side-look camera (_0)

### Camera 2 - Front-look camera (_1)

In [3]:
# Load previously saved intrinsic and extrinsic matrices 
frontlook_mtx = np.load(path+'June\\frontlook\\calibrate\\frontlook_camera_matrix.npy')
frontlook_dist = np.load(path+'June\\frontlook\\calibrate\\frontlook_distortion_matrix.npy')
frontlook_rvecs = np.load(path+'June\\frontlook\\calibrate\\frontlook_rotation_matrix.npy')
frontlook_tvecs = np.load(path+'June\\frontlook\\calibrate\\frontlook_translation_matrix.npy')

print ('Computed camera matrix\n', frontlook_mtx)
print ('Computed distortion matrix\n', frontlook_dist)
print ('Computed rotation vector\n', frontlook_rvecs)
print ('Computed translation vector\n', frontlook_tvecs)

rotM = cv2.Rodrigues(frontlook_rvecs)[0]

Computed camera matrix
 [[560.02665252   0.         402.60078829]
 [  0.         869.92913382 252.54590069]
 [  0.           0.           1.        ]]
Computed distortion matrix
 [[-4.70342049e-01  9.05259576e+00  4.59566683e-03  2.16735826e-02
  -5.32980337e+01]]
Computed rotation vector
 [[ 0.04456356]
 [ 3.01837349]
 [-0.90960737]]
Computed translation vector
 [[  31.90938981]
 [-232.49838609]
 [ 249.25996024]]


In [4]:
#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

In [5]:
lower_body_indices = list(range(9, 15)) + list(range(19, 25))
print (len(lower_body_indices))

#Dimensions of the image 
w = 800
h = 448

order = ['right hip', 'right knee', 'right ankle', 'left hip', 'left knee', 'left ankle', 'left toe 1', 'left toe 2', \
         'left heel', 'right toe 1', 'right toe 2', 'right heel']

12


In [6]:
def convert2d_to_3d(coord, scale, shift_array):
    frontlook_tvecs1 = frontlook_tvecs-np.array(shift_array)
    col = coord[0]
    row = coord[1]
    u=col-w/2
    v=h/2-row
    p = np.matmul(np.linalg.pinv(frontlook_mtx), np.array([col, row, 1]))
    return (np.matmul(np.matrix(rotM).T, (scale*p.reshape(-1,1) - np.matrix(frontlook_tvecs1))).T)

In [32]:
raw_path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\openpose_data' 
for cohort in cohorts[3:]:
    for trial in trials[1:]:
        for camera in cameras[1:]: #Front look camera only
            frame_path = raw_path+cohort+trial+camera #Path to save the frames to 
            videos = os.listdir(frame_path)
#             print (len(videos))
            for video in videos[5:]:
                frames = glob.glob(frame_path+video+'\\processed2d\\*.csv')
                frames_selected = [frames[0], frames[3], frames[5], frames[9], frames[15], \
                                   frames[21], frames[27], frames[31], frames[35], frames[40], frames[45]]
                for frame in frames_selected:
                    data_frame = pd.DataFrame()
#                     im_path = frame_path+video+'\\'+frame
                    try:
                        frame_csv = pd.read_csv(frame, index_col = 0)
#                         print (frame_csv)
                        coords = frame_csv[['x', 'y']].values
                        #Only reading the x, y coordinates for 12 locations 
                        array = np.array([[5], [-120], [-530]]) #np.array([[5], [-60], [-500]])
                        data_frame=pd.DataFrame(columns = ['x', 'y', 'z'])
                        for idx, coord in enumerate(coords):
                            if idx in [0, 3]: #Hip 
                                scale = [[(37+65+300)], [(37+65+500)], [(37+65+500)]] #[[(37+65+300)], [(37+65+900)], [(37+65+510)]]
                            elif idx in [1, 4]: #Knee
                                scale = [[(37+65+300)], [(37+65+700)], [(37+65+520)]] #[[(37+65+300)], [(37+65+1300)], [(37+65+550)]]
                            else: #Ankle, Toes and Heel
                                scale = [[(37+65+300)], [(37+65+300)], [(37+65+550)]] #[[(37+65+300)], [(37+65+1000)], [(37+65+590)]] #[[(37+65+300)], [(37+65+300)], [(37+65+550)]] #[[(37+65+300)], [(37+65+2700)], [(37+65+580)]]
                            data_frame.loc[len(data_frame)] = np.array(convert2d_to_3d(coord, scale, array))[0].T
                        data_frame.index = order
                        data_frame['confidence'] = frame_csv['confidence']
                        print (frame)
                        print (data_frame)
#                         data_frame.to_csv(frame[:-4]+'_3dcheck.csv')
                    except Exception as e:
                        print (e)

C:\Users\purpl\Box\Gait Video Project\GaitVideoData\video\openpose_data\ExtraHOA\walking\lower_body\InkedGVS_124_W_T1_1_Trim\processed2d\0.csv
                     x           y           z  confidence
right hip    54.233192  148.476691  114.407657    0.679501
right knee   54.830383  175.071351   72.781771    0.781446
right ankle  51.379007  182.818410   31.640879    0.811414
left hip     22.761787  149.936548  113.343769    0.681224
left knee    27.627027  173.849293   73.508943    0.803639
left ankle   35.710563  185.060042   30.106922    0.847228
left toe 1   38.604000  192.505579   25.179566    0.855304
left toe 2   31.578963  191.759927   25.652474    0.899367
left heel    37.556613  187.358793   28.588547    0.693563
right toe 1  49.929363  190.836324   26.320799    0.784095
right toe 2  56.034516  189.743116   27.064197    0.747329
right heel   49.688593  184.721730   30.373871    0.575129
C:\Users\purpl\Box\Gait Video Project\GaitVideoData\video\openpose_data\ExtraHOA\walking\l